In [1]:
!pip install transformers
!pip install sentencepiece
!pip install sacremoses
import pandas as pd
import json
import tensorflow as tf
import numpy as np
from transformers import *
from tqdm import tqdm
import os
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 4.2 MB 4.8 MB/s 
     |████████████████████████████████| 6.6 MB 28.7 MB/s 
     |████████████████████████████████| 84 kB 2.3 MB/s 
     |████████████████████████████████| 596 kB 24.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.7 MB/s 
     |████████████████████████████████| 880 kB 5.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c018a600c00bfdbd2412e8a42ef6546e1f859272488b59885ec031d2907605a1
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Mounted at /content/drive


In [2]:
data = json.load(open('/content/drive/MyDrive/NLP/NXMP1902008040.json', encoding='UTF-8'))

In [3]:
df = pd.DataFrame(data["document"])
del data

In [4]:
pumsa=df['sentence']
pumsa

0       [{'id': 'NWRW1800000022.417.1.1', 'form': '[제주...
1       [{'id': 'NWRW1800000024.321.1.1', 'form': '이바노...
2       [{'id': 'NWRW1800000022.81.1.1', 'form': '한국화학...
3       [{'id': 'NWRW1800000021.156.1.1', 'form': '[강원...
4       [{'id': 'NWRW1800000022.97.1.1', 'form': '모든 일...
                              ...                        
7260    [{'id': 'NWRW1800000045.95.1.1', 'form': 'GS그룹...
7261    [{'id': 'NWRW1800000053.309.1.1', 'form': '김무성...
7262    [{'id': 'NWRW1800000029.59.1.1', 'form': '도요타 ...
7263    [{'id': 'NWRW1800000024.69.1.1', 'form': '용산 범...
7264    [{'id': 'NWRW1800000024.432.1.1', 'form': '교보악...
Name: sentence, Length: 7265, dtype: object

In [5]:
#pumsa[0][1]

In [6]:
#pumsa[0][1]['morpheme'][0]['label']

In [7]:
form_list=[]
form_index=[]
for i in range(len(pumsa)):
    for j in range(len(pumsa[i])):
        for k in range(len(pumsa[i][j]['morpheme'])):
            form_list.append(pumsa[i][j]['morpheme'][k]['form'])
            form_index.append(k+1)

In [8]:
#form_list

In [9]:
#form_index

In [10]:
label_list=[]
for i in range(len(pumsa)):
    for j in range(len(pumsa[i])):
        for k in range(len(pumsa[i][j]['morpheme'])):
            label_list.append(pumsa[i][j]['morpheme'][k]['label'])

In [11]:
#label_list

In [12]:
raw_data=pd.DataFrame()
raw_data['index']=form_index
raw_data['word']=form_list
raw_data['label']=label_list
del df

In [13]:
#raw_data

In [14]:
#raw_data.loc[raw_data['word']=='.']

In [15]:
raw_data['word'] = raw_data['word'].astype(str)
raw_data['label'] = raw_data['label'].astype(str)

raw_data['word'] = raw_data['word'].str.replace(r'[^ㄱ-ㅣ가-힣0-9a-zA-Z.]+', "", regex=True)#ㄱ-ㅣ가-힣0-9a-zA-Z로 시작하지 않는게 한번 이상 있으면, 공백으로 바꾼다

In [16]:
pumsa_data = [list(x) for x in raw_data[['index', 'word', 'label']].to_numpy()]

In [17]:
#print(pumsa_data[:20])

In [18]:
p_label = raw_data['label'].unique().tolist()
p_label_dict = {word:i for i, word in enumerate(p_label)}
p_label_dict.update({"[PAD]":len(p_label_dict)})
p_index_to_ner = {i:j for j, i in p_label_dict.items()}

In [19]:
#print(p_label_dict)

In [20]:
print(p_index_to_ner)

{0: 'SS', 1: 'NNP', 2: 'SP', 3: 'NNG', 4: 'VV', 5: 'EC', 6: 'SN', 7: 'NNB', 8: 'EP', 9: 'EF', 10: 'SL', 11: 'JKB', 12: 'XSV', 13: 'ETM', 14: 'JKS', 15: 'JKO', 16: 'SF', 17: 'JX', 18: 'JKQ', 19: 'JC', 20: 'MAG', 21: 'JKG', 22: 'XSN', 23: 'NP', 24: 'VCP', 25: 'MMN', 26: 'VA', 27: 'SO', 28: 'SW', 29: 'VX', 30: 'NR', 31: 'ETN', 32: 'XPN', 33: 'MMD', 34: 'SH', 35: 'XSA', 36: 'MAJ', 37: 'SE', 38: 'MMA', 39: 'JKC', 40: 'VCN', 41: 'NA', 42: 'XR', 43: 'IC', 44: 'NF', 45: 'JKV', 46: 'NV', 47: '[PAD]'}


In [21]:
p_tups = []
p_temp_tup = []
p_temp_tup.append(pumsa_data[0][1:])
p_sentences = []
p_targets = []
for i, j, k in pumsa_data:
  
  if i != 1:
    p_temp_tup.append([j,p_label_dict[k]])
  if i == 1:
    if len(p_temp_tup) != 0:
      p_tups.append(p_temp_tup)
      p_temp_tup = []
      p_temp_tup.append([j,p_label_dict[k]])
# tups[0]는 첫번째 문장에 있는 단어들+개체번호, tups[1]은 두번째 문장에 있는 단어들+개체번호, ...
p_tups.pop(0)

[['', 'SS']]

In [22]:
#print(p_tups[0], p_tups[1])

In [23]:
p_sentences = []
p_targets = []
for p_tup in p_tups:
  p_sentence = []
  p_target = []
  p_sentence.append("[CLS]")
  p_target.append(p_label_dict['SW'])
  for i, j in p_tup:
    p_sentence.append(i)
    p_target.append(j)
  p_sentence.append("[SEP]")
  p_target.append(p_label_dict['SW'])
  p_sentences.append(p_sentence)
  p_targets.append(p_target)

In [24]:
#p_sentences[0]

In [25]:
#p_targets[0]

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpehvz9hqg


Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
creating metadata file for /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpsbqccemk


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://hug

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
creating metadata file for /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidde

In [27]:
def tokenize_and_preserve_labels(sentence, text_labels):
  tokenized_sentence = []
  labels = []

  for word, label in zip(sentence, text_labels):

    tokenized_word = tokenizer.tokenize(word)
    n_subwords = len(tokenized_word)

    tokenized_sentence.extend(tokenized_word)
    labels.extend([label] * n_subwords)

  return tokenized_sentence, labels

In [28]:
p_tokenized_texts_and_labels = [
                              tokenize_and_preserve_labels(sent, labs)
                              for sent, labs in zip(p_sentences, p_targets)]

In [29]:
#print(p_tokenized_texts_and_labels[:2])
# [(문장, 개체들), (문장, 개체들),...] 형식으로 저장되어 있음.

In [30]:
p_tokenized_texts = [token_label_pair[0] for token_label_pair in p_tokenized_texts_and_labels]
p_labels = [token_label_pair[1] for token_label_pair in p_tokenized_texts_and_labels]

In [31]:
#p_tokenized_texts[1]

In [32]:
#p_labels[1]

In [33]:
print(np.quantile(np.array([len(x) for x in p_tokenized_texts]), 0.975))
max_len = 100
bs = 1

100.0


In [34]:
p_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in p_tokenized_texts],
                          maxlen=max_len, dtype = "int", value=tokenizer.convert_tokens_to_ids("[PAD]"), truncating="post", padding="post")

In [35]:
#p_input_ids[1]

In [36]:
p_tags = pad_sequences([lab for lab in p_labels], maxlen=max_len, value=p_label_dict["[PAD]"], padding='post',\
                     dtype='int', truncating='post')

In [37]:
#p_tags[1]

In [38]:
p_attention_masks = np.array([[int(i != tokenizer.convert_tokens_to_ids("[PAD]")) for i in ii] for ii in p_input_ids])
p_attention_masks[1]
# 어텐션 마스크란 BERT가 어텐션 연산을 할때, 불필요하게 패딩토큰에 대해 어텐션을 하지 않게, 실제 토큰과 패딩을 구분해준다 : 실제 - 1, 패딩 - 0

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [39]:
p_tr_inputs, p_val_inputs, p_tr_tags, p_val_tags = train_test_split(p_input_ids, p_tags,
                                                            random_state=2018, test_size=0.1)

In [40]:
p_tr_masks, p_val_masks, _, _ = train_test_split(p_attention_masks, p_input_ids,
                                             random_state=2018, test_size=0.1)

In [41]:
# TPU 작동을 위해 실행
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
# TPU 초기화 하는 코드

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.115.62.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.115.62.234:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [42]:
SEQ_LEN = max_len
def create_model():
  model = TFBertModel.from_pretrained("bert-base-multilingual-cased", from_pt=True, num_labels=len(p_label_dict), output_attentions = False,
    output_hidden_states = False)

  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids') # 토큰 인풋
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks') # 마스크 인풋

  bert_outputs = model([token_inputs, mask_inputs])
  bert_outputs = bert_outputs[0] # shape : (Batch_size, max_len, 30(개체의 총 개수))
  nr = tf.keras.layers.Dense(48, activation='softmax')(bert_outputs) # shape : (Batch_size, max_len, 30)
  
  nr_model = tf.keras.Model([token_inputs, mask_inputs], nr)
  
  nr_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
      metrics=['sparse_categorical_accuracy'])
  nr_model.summary()
  return nr_model

In [43]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
# TPU를 활용하기 위해 context로 묶어주기
with strategy.scope():
  p_nr_model = create_model()
  p_nr_model.fit([p_tr_inputs, p_tr_masks], p_tr_tags, validation_data=([p_val_inputs, p_val_masks], p_val_tags), epochs=1, shuffle=False, batch_size=bs)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
creating metadata file for /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
loading weights file https://huggingface.co/bert-base-multilingual-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
Loading PyTorch weights from /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
PyTorch checkpoint contains 270,378,749 parameters


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  177853440   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]']            
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


135072/135072 [==============================] - 11035s 81ms/step - loss: 0.0292 - sparse_categorical_accuracy: 0.9910 - val_loss: 0.0174 - val_sparse_categorical_accuracy: 0.9949


In [44]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [45]:
p_y_predicted = p_nr_model.predict([p_val_inputs, p_val_masks])

In [46]:
p_f_label = [i for i, j in p_label_dict.items()]
p_val_tags_l = [p_index_to_ner[x] for x in np.ravel(p_val_tags).astype(int).tolist()]
p_y_predicted_l = [p_index_to_ner[x] for x in np.ravel(np.argmax(p_y_predicted, axis=2)).astype(int).tolist()] #axis 2일경우 z축 기준
p_f_label.remove("[PAD]")

In [47]:
print(classification_report(p_val_tags_l, p_y_predicted_l, labels=p_f_label))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

              precision    recall  f1-score   support

          SS       0.00      0.00      0.00         1
         NNP       0.98      0.95      0.96     53605
          SP       1.00      0.99      0.99       669
         NNG       0.99      0.99      0.99    250245
          VV       0.99      0.99      0.99     41456
          EC       0.99      0.98      0.99     26570
          SN       1.00      1.00      1.00     14383
         NNB       0.99      0.99      0.99     19125
          EP       1.00      1.00      1.00     10333
          EF       0.98      0.99      0.99     16624
          SL       0.99      1.00      0.99      2764
         JKB       0.99      1.00      0.99     20204
         XSV       0.99      0.99      0.99     14997
         ETM       0.99      1.00      0.99     22107
         JKS       0.99      0.99      0.99     10539
         JKO       1.00      1.00      1.00     15769
          SF       1.00      1.00      1.00     12899
          JX       0.99    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
def ner_inference(test_sentence):
  

  tokenized_sentence = np.array([tokenizer.encode(test_sentence, max_length=max_len, truncation=True, padding='max_length')])
  tokenized_mask = np.array([[int(x!=1) for x in tokenized_sentence[0].tolist()]])
  ans = p_nr_model.predict([tokenized_sentence, tokenized_mask])
  ans = np.argmax(ans, axis=2)

  tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, ans[0]):
    if (token.startswith("##")):
      new_labels.append(p_index_to_ner[label_idx])
      new_tokens.append(token[2:])
    elif (token=='[CLS]'):
      pass
    elif (token=='[SEP]'):
      pass
    elif (token=='[PAD]'):
      pass
    elif (token != '[CLS]' or token != '[SEP]'):
      new_tokens.append(token)
      new_labels.append(p_index_to_ner[label_idx])
  
  
  for token, label in zip(new_tokens, new_labels):
      print("{}\t{}".format(label, token))
  
  print("\n") 

In [49]:
ner_inference("안녕하세요 저는 서울시립대학교 전자전기컴퓨터공학부 최윤석입니다.")

NNP	안
NNP	녕
NNP	하
NNP	세
NNP	요
NP	저
JX	는
NNP	서울
NNP	시
NNP	립
NNP	대학교
NNP	전
NNP	자
NNP	전
NNP	기
NNP	컴
NNP	퓨
NNP	터
NNP	공
NNP	학
NNP	부
NNP	최
NNP	윤
NNP	석
EF	입
EF	니다
SF	.




In [50]:
ner_inference("최윤석은 어제 제주도에 놀러가서 화강암을 구경하고 왔습니다")

NNP	최
NNP	윤
NNP	석
NNP	은
MAG	어
MAG	제
NNP	제
NNP	주
NNP	도에
NNG	놀
NNG	러
NNG	가
NNG	서
NNP	화
NNP	강
NNP	암
NNP	을
VA	구
VA	경
NNG	하고
VV	왔
EF	습
EF	니다




In [52]:
from keras.models import load_model

p_nr_model.save('pumsa_batch1.h5')